<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Code_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models, similarities
from gensim.models import Word2Vec

In [2]:
max_seq_length = 100

In [3]:
ratings1 = pd.read_csv('/content/drive/My Drive/Movielensdata/ml25m/ratings.csv')
ratings = ratings1.iloc[0:1000,:]
#print(len(ratings['movieId'].unique()))
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


**Sorting the dataframe**

In [4]:
ratings.sort_values(by=['userId','timestamp'],inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
ratings.head(10)

,userId,movieId,rating,timestamp
36,1,5952,4.0,1147868053
13,1,2012,2.5,1147868068
12,1,2011,2.5,1147868079
11,1,1653,4.0,1147868097
9,1,1250,4.0,1147868414
40,1,6539,3.5,1147868461
39,1,6377,4.0,1147868469
21,1,3448,4.0,1147868480
5,1,1088,4.0,1147868495
4,1,899,3.5,1147868510


In [6]:
users_total = len(ratings['userId'].unique())
print(users_total)

4


**Selecting the most recent movies rated by each user and padding if necessary**

In [7]:
movie_list = []
for i in range(users_total):
  list1 = []
  list1 = ratings.loc[ratings['userId'] ==(i+1),['movieId']]['movieId'].tolist()
  if len(list1)>max_seq_length:
    list1 = list1[(len(list1)-max_seq_length):]
  elif len(list1)<max_seq_length:
    for j in range((max_seq_length-len(list1))):
      list1.append(0)
  movie_list.append(list1)
#print(movie_list)

**Selecting the most recent ratings of each user and padding if necessary**

In [8]:
rating_list =[]
for i in range(users_total):
  list2 = []
  list2 = ratings.loc[ratings['userId'] ==(i+1),['rating']]['rating'].tolist()
  if len(list2)>max_seq_length:
    list2 = list2[(len(list2)-max_seq_length):]
  elif len(list2)<max_seq_length:
    for j in range((max_seq_length-len(list2))):
      list2.append(0)
  rating_list.append(list2)
#print(rating_list)

In [9]:
movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(users_total)])
movies_transpose.index.names = ['userId']
print(movies_transpose)

            0     1     2       3     4   ...     95      96     97    98    99
userId                                    ...                                  
1         5952  2012  2011    1653  1250  ...      0       0      0     0     0
2         4306  8368  8360    3793  4995  ...   2987    1293   7090  1136  2150
3       176371  3968  8983  102407  6548  ...  88125  106489  81834  4262  1732
4         8665  8961  4993    1580  5444  ...      0       0      0     0     0

[4 rows x 100 columns]


In [10]:
ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(users_total)])
ratings_transpose.index.names = ['userId']
print(ratings_transpose)

         0    1    2    3    4    5    6   ...   93   94   95   96   97   98   99
userId                                     ...                                   
1       4.0  2.5  2.5  4.0  4.0  3.5  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2       4.5  5.0  4.0  5.0  5.0  4.0  5.0  ...  0.5  3.0  0.5  5.0  2.0  5.0  5.0
3       4.5  3.0  4.0  4.0  3.0  4.0  3.5  ...  2.5  4.0  3.5  3.5  3.5  4.5  4.5
4       3.5  5.0  4.5  4.5  4.0  4.0  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[4 rows x 100 columns]


**Separating each row as a list of strings into a list of lists 'sent' for gensim imput format**

In [11]:
# Select features from original dataset to form a new dataframe 
df1 = movies_transpose.iloc[:]# For each row, combine all the columns into one column
df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)# Store them in a pandas dataframe
df_clean = pd.DataFrame({'clean': df2})# Create the list of list format of the custom corpus for gensim modeling 
sent = [row.split(',') for row in df_clean['clean']]

In [12]:
#list of lists format of the movies transpose matrix
print(sent)

[['5952', '2012', '2011', '1653', '1250', '6539', '6377', '3448', '1088', '899', '4308', '2161', '6711', '3949', '8360', '5878', '306', '1175', '307', '1237', '7327', '8154', '7234', '2843', '4144', '7365', '2068', '4422', '4973', '6016', '8873', '2692', '27721', '7323', '6954', '8014', '7939', '6370', '8973', '4703', '31956', '5147', '8786', '1260', '2351', '7940', '7209', '8685', '7820', '7937', '7938', '8405', '4325', '2632', '1217', '8729', '5912', '5767', '665', '2573', '27266', '8327', '32591', '5269', '3569', '27193', '5684', '7318', '296', '7361', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['4306', '8368', '8360', '3793', '4995', '4963', '8636', '2355', '8665', '2571', '589', '33493', '7153', '1873', '1246', '1584', '3994', '2139', '31923', '4720', '2294', '2745', '2138', '858', '33660', '2268', '2501', '6539', '1270', '6565', '4535', '1198', '1302', '349', '3098', '1907'

In [13]:
model = Word2Vec(sent,min_count=1,size= 50,workers=3, window =3, sg = 1)

In [14]:
model.save('wordtovec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [15]:
#user_vec = []
#for i in range(users_total):
#  a = np.zeros((50,))
#  for j in range(max_seq_length):
#    a = a+ model[sent[i][j]]
#  user_vec.append(a)

In [16]:
#user_vec

**Corpus format for LSI model**

In [17]:
corpus = gensim.matutils.Dense2Corpus(np.array(sent),documents_columns=False)
print(list(corpus))
total_topics = 10

[[(0, 5952.0), (1, 2012.0), (2, 2011.0), (3, 1653.0), (4, 1250.0), (5, 6539.0), (6, 6377.0), (7, 3448.0), (8, 1088.0), (9, 899.0), (10, 4308.0), (11, 2161.0), (12, 6711.0), (13, 3949.0), (14, 8360.0), (15, 5878.0), (16, 306.0), (17, 1175.0), (18, 307.0), (19, 1237.0), (20, 7327.0), (21, 8154.0), (22, 7234.0), (23, 2843.0), (24, 4144.0), (25, 7365.0), (26, 2068.0), (27, 4422.0), (28, 4973.0), (29, 6016.0), (30, 8873.0), (31, 2692.0), (32, 27721.0), (33, 7323.0), (34, 6954.0), (35, 8014.0), (36, 7939.0), (37, 6370.0), (38, 8973.0), (39, 4703.0), (40, 31956.0), (41, 5147.0), (42, 8786.0), (43, 1260.0), (44, 2351.0), (45, 7940.0), (46, 7209.0), (47, 8685.0), (48, 7820.0), (49, 7937.0), (50, 7938.0), (51, 8405.0), (52, 4325.0), (53, 2632.0), (54, 1217.0), (55, 8729.0), (56, 5912.0), (57, 5767.0), (58, 665.0), (59, 2573.0), (60, 27266.0), (61, 8327.0), (62, 32591.0), (63, 5269.0), (64, 3569.0), (65, 27193.0), (66, 5684.0), (67, 7318.0), (68, 296.0), (69, 7361.0)], [(0, 4306.0), (1, 8368.0), 

In [18]:
lsi = models.LsiModel(corpus, num_topics=total_topics)
index = similarities.MatrixSimilarity(lsi[corpus])
lsi_data = lsi[corpus]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [19]:
#list(lsi_data)

In [20]:
lsi_topics = lsi.print_topics()
for topic in lsi_topics:
  print(topic)

(0, '-0.185*"8" + -0.184*"33" + -0.177*"26" + -0.174*"0" + -0.173*"79" + -0.171*"13" + -0.166*"28" + -0.166*"87" + -0.157*"41" + -0.156*"71"')
(1, '0.417*"48" + 0.369*"50" + 0.349*"40" + 0.338*"23" + 0.321*"21" + 0.307*"28" + 0.226*"7" + 0.214*"13" + 0.096*"36" + -0.095*"79"')
(2, '-0.352*"72" + -0.349*"18" + -0.345*"66" + -0.328*"83" + -0.327*"24" + -0.326*"11" + -0.310*"89" + -0.294*"88" + 0.100*"33" + -0.095*"1"')
(3, '0.424*"62" + 0.369*"60" + 0.368*"65" + 0.349*"32" + 0.321*"40" + -0.176*"23" + -0.166*"28" + -0.145*"13" + -0.136*"7" + 0.116*"14"')


In [21]:
lsi.save('lsimodel')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Corpus format for TF-IDF model**

In [22]:
tfidf = models.TfidfModel(corpus)
tfidf_data = tfidf[corpus]
print(list(tfidf_data))

[[], [(70, 0.032610107927719026), (71, 0.12383270318631072), (72, 0.4953861075446116), (73, 0.038167235264278176), (74, 0.013173985949604165), (75, 0.02658297479652551), (76, 0.09603324280367276), (77, 0.043517007103204525), (78, 0.02879476677127542), (79, 0.11072783573591749), (80, 0.09900533826974296), (81, 0.012634861655758875), (82, 0.055612744465118116), (83, 0.45847682896597247), (84, 0.046945284664066896), (85, 0.021108789659019477), (86, 0.03212627843324248), (87, 0.01756992249941962), (88, 0.42643349273178316), (89, 0.4250649464474066), (90, 0.0177859969967878), (91, 0.13665907430303434), (92, 0.16233885648378976), (93, 0.008693155835508644), (94, 0.04879491685448338), (95, 0.0994883390063767), (96, 0.04306609385177271), (97, 0.23614741330941105), (98, 0.03783687750627517), (99, 0.07161028753388346)], [(70, 0.038816128837187776), (71, 0.19763763930471132), (72, 0.10570280496261183), (73, 0.10362530099775034), (74, 0.09849681275415151), (75, 0.07960657023942645), (76, 0.1308081

In [23]:
#lsi_list = np.zeros((len(lsi_data), len(lsi_topics)), dtype=np.float64)
#for i in range(len(corpus)):
#    for j in range(len(lsi_topics)):
#        lsi_list[i][j]= list(lsi_data)[i][j][1]

tfidf_token= np.zeros((len(tfidf_data), 350), dtype=np.float64)
tfidf_vals= np.zeros((len(tfidf_data), 350), dtype=np.float64)

for i in range(len(corpus)):
    for k in range(len(list(tfidf_data)[i])):
        tfidf_token[i][k]=(list(tfidf_data))[i][k][0]
        tfidf_vals[i][k]=(list(tfidf_data))[i][k][1]

tfidf_list=list(tfidf_data)

In [24]:
tfidf_list[1]

[(70, 0.032610107927719026),
 (71, 0.12383270318631072),
 (72, 0.4953861075446116),
 (73, 0.038167235264278176),
 (74, 0.013173985949604165),
 (75, 0.02658297479652551),
 (76, 0.09603324280367276),
 (77, 0.043517007103204525),
 (78, 0.02879476677127542),
 (79, 0.11072783573591749),
 (80, 0.09900533826974296),
 (81, 0.012634861655758875),
 (82, 0.055612744465118116),
 (83, 0.45847682896597247),
 (84, 0.046945284664066896),
 (85, 0.021108789659019477),
 (86, 0.03212627843324248),
 (87, 0.01756992249941962),
 (88, 0.42643349273178316),
 (89, 0.4250649464474066),
 (90, 0.0177859969967878),
 (91, 0.13665907430303434),
 (92, 0.16233885648378976),
 (93, 0.008693155835508644),
 (94, 0.04879491685448338),
 (95, 0.0994883390063767),
 (96, 0.04306609385177271),
 (97, 0.23614741330941105),
 (98, 0.03783687750627517),
 (99, 0.07161028753388346)]

**Tagged data format for Doc2Vec model**

In [25]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = []
tags = []
test_data = []
test_data = test_data + sent[3]
for i in range(3):
  tagged_data = tagged_data + [TaggedDocument(words=sent[i], tags=[str(i)])]
print(tagged_data)
print(test_data)
#model = Doc2Vec(documents=tagged_data, vector_size=5, window=1, min_count=1, workers=1)

[TaggedDocument(words=['5952', '2012', '2011', '1653', '1250', '6539', '6377', '3448', '1088', '899', '4308', '2161', '6711', '3949', '8360', '5878', '306', '1175', '307', '1237', '7327', '8154', '7234', '2843', '4144', '7365', '2068', '4422', '4973', '6016', '8873', '2692', '27721', '7323', '6954', '8014', '7939', '6370', '8973', '4703', '31956', '5147', '8786', '1260', '2351', '7940', '7209', '8685', '7820', '7937', '7938', '8405', '4325', '2632', '1217', '8729', '5912', '5767', '665', '2573', '27266', '8327', '32591', '5269', '3569', '27193', '5684', '7318', '296', '7361', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], tags=['0']), TaggedDocument(words=['4306', '8368', '8360', '3793', '4995', '4963', '8636', '2355', '8665', '2571', '589', '33493', '7153', '1873', '1246', '1584', '3994', '2139', '31923', '4720', '2294', '2745', '2138', '858', '33660', '2268', '2501', '6539', '1270'

In [26]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [27]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])

V1_infer [ 0.09125655  0.03866785 -0.02686151 -0.07190301  0.05158131 -0.07877599
  0.0292795  -0.19514167  0.03184161  0.11824249  0.00481782  0.03587288
 -0.02867902  0.06504951  0.15659502  0.03809637  0.02560205  0.04049287
  0.08931653  0.03504137]
[('2', 0.6066149473190308), ('0', 0.5754348635673523)]
[-0.12543246 -7.916983   -0.85310525 -1.6596771  -1.2856377   0.09855106
  0.74106675 -4.9402575   8.841046    3.6958816  -0.38346356 -3.7024946
  9.660955   -6.573852    2.976019   -5.8956704  -4.7855134   8.85131
 -4.029579   -9.443697  ]


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
